In [ ]:
%load_ext autoreload
%autoreload 2
%matplotlib widget

In [ ]:
import json
import numpy as np
import matplotlib.pyplot as plt

from mpl_toolkits.mplot3d import axes3d
from matplotlib import cm
from scipy.interpolate import griddata

from pathlib import Path

In [ ]:
paths = []
results = {}

# paths.append(Path("../../results/distances/20220807-185201.json")) # 3000 normal phi

# paths.append(Path("../../results/distances/20220807-201939.json")) # 500 normal phi
# paths.append(Path("../../results/distances/20220807-215412.json")) # 500 big phi

# paths.append(Path("../../results/distances/20220808-105411.json")) # 1500 small phi
# paths.append(Path("../../results/distances/20220807-164740.json")) # 1500 normal phi
# paths.append(Path("../../results/distances/20220808-100633.json")) # 1500 big phi

# paths.append(Path("../../results/distances/20220808-133109.json")) # 1500 T_half
# paths.append(Path("../../results/distances/20220808-134307.json")) # 1500 T_half long

# paths.append(Path("../../results/distances/20220808-142650.json")) # 1500 mix phi/T_half
# paths.append(Path("../../results/distances/20220808-171339.json")) # 1500 mix phi/T_half
# paths.append(Path("../../results/distances/20220808-184217.json")) # 1500 mix phi/T_half
#paths.append(Path("../../results/distances/20220808-214430.json"))
paths.append(Path("../../results/distances/20220809-112831.json"))
for path in paths:
    with path.open() as f:
        result = json.load(f)
        results.update(result)

In [ ]:
# 3D plots
parameter_1 = "T_half"
parameter_2 = "beta"
data = [(v["parameters"]["cx"]["params"][parameter_1],v["parameters"]["cx"]["params"][parameter_2],v["mean_distance"]) for v in results.values()]
data = sorted(data)


minimum = zip(min(data, key=lambda tup: tup[2]))
x,y,z = zip(*data)

x = np.log10(x)

grid_x,grid_y = np.mgrid[min(x):max(x):100j, min(y):max(y):100j]
#grid_x,grid_y = np.meshgrid(np.linspace(np.log10(min(x)), np.log10(max(x)), 100), np.linspace(min(y), max(y), 100))
grid_z = griddata((x,y),z,(grid_x,grid_y),method='nearest',rescale=True) # method can be nearest, linear or cubic

plt.figure(figsize=(7,7))
ax = plt.axes(projection='3d', computed_zorder=False)
ax.plot_surface(grid_x,grid_y,grid_z,cmap=cm.Spectral)
ax.set_xlabel("$T_{1/2}$")
ax.set_ylabel("$\\beta$")


#ax.scatter(*zip(*data))
#ax.scatter(*minimum, alpha=1, s=100, marker="o")

#ax.set_xscale("log")
plt.show()

In [ ]:
from pim.setup import enumerate_results, load_results
from tqdm import tqdm

file = "../../results/dye-stability_20220808-173622"
paths = enumerate_results([file])

beta_min = 0.18
beta_max = 0.22
log_t_half_min = 3.6
log_t_half_max = 4.0

for result in load_results(tqdm(paths)):
    params = result.parameters["cx"]["params"]
    if beta_min <= params["beta"] <= beta_max and log_t_half_min <= np.log10(params["T_half"]) <= log_t_half_max:
        found = result
        break


In [ ]:
print(found.parameters)
print(np.linalg.norm(found.closest_position()))

fig = plt.figure()
found.plot_path(plt.axes())
#print(found.recordings)